In [19]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, execute

Implementation of grover's quantum algorithm(O(sqrt(n)):

In [20]:
flipper=QuantumCircuit(2,name='flipper')
flipper.cz(0,1)
flipper.to_gate()
flipper.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [21]:
hadam=QuantumCircuit(2,2)
hadam.h([0,1])
hadam.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘
c_0: ═════
          
c_1: ═════

In [26]:
hadam.append(flipper,[1,0])
hadam.draw()

┌───┐┌──────────┐
q_0: ┤ H ├┤1         ├
     ├───┤│  flipper │
q_1: ┤ H ├┤0         ├
     └───┘└──────────┘
c_0: ═════════════════
                      
c_1: ═════════════════

In [27]:
back=Aer.get_backend('statevector_simulator')
execi=execute(hadam,back)
vec=execi.result().get_statevector()
print(vec)

Statevector([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j],
            dims=(2, 2))


maths: |w>=np.array(0,0,0,1) and |s>=0.5*np.array(1,1,1,1)
In the span of |w> and |s> exists |s'>=1/sqrt(3)*np.array(1,1,1,0) orthogonal to |w>
Now, flipper(|s>)->0.5*np.array(1,1,1,-1) so |s> is reflected about |s'> by the winning probability element.
To get closer to |w>(the vector we want), we apply reflection:
refl(|s>)=|s><s|-I. This algo is iterative.

In [32]:
refl=QuantumCircuit(2,name='refl')
refl.h([0,1])
refl.z([0,1])
refl.cz(0,1)
refl.h([0,1])
refl.to_gate()
refl.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [44]:
final_circ=QuantumCircuit(2,2)
final_circ.h([0,1])
final_circ.append(flipper,[0,1])
final_circ.append(refl,[0,1])
final_circ.measure([0,1],[0,1])
final_circ.draw()

┌───┐┌──────────┐┌───────┐┌─┐   
q_0: ┤ H ├┤0         ├┤0      ├┤M├───
     ├───┤│  flipper ││  refl │└╥┘┌─┐
q_1: ┤ H ├┤1         ├┤1      ├─╫─┤M├
     └───┘└──────────┘└───────┘ ║ └╥┘
c_0: ═══════════════════════════╩══╬═
                                   ║ 
c_1: ══════════════════════════════╩═

In [48]:
new_back=Aer.get_backend('qasm_simulator')
ex=execute(final_circ,new_back,shots=1)
result=ex.result()
result.get_counts()

{'11': 1}